# Output table for Maria

Maria wants me to output a table of the raw counts for her husband to look at.

In [1]:
import os
import sys
import re
from pathlib import Path

from IPython.display import display, HTML, Markdown
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Project level imports
from larval_gonad.notebook import Nb

In [2]:
# Setup notebook
nbconfig = Nb.setup_notebook(seurat_dir='../output/scrnaseq-wf/scrnaseq_combine_force')

last updated: 2018-09-13 
Git hash: 42b50b2bae7773307f49194899ffbeb2475982b9


In [3]:
# get raw counts table
raw = nbconfig.seurat.get_raw()

# get cluster info
clusters = nbconfig.seurat.get_clusters('res.0.6')
clusters = clusters.map(nbconfig.cluster_annot)

# merge together
raw_w_meta = raw.T.join(clusters)

In [4]:
# pull out gonia and early
gonia = raw_w_meta.query('cluster == "Spermatogonia"').drop('cluster', axis=1).T
gonia.sort_index(axis=1, inplace=True)
gonia.index.name = 'FBgn'

early = raw_w_meta.query('cluster == "Early 1º Spermatocytes"').drop('cluster', axis=1).T
early.sort_index(axis=1, inplace=True)
early.index.name = 'FBgn'

In [48]:
# export the data
gonia.to_csv('../output/2018-09-12_output_table_for_maria_gonia.csv.gz', compression='gzip')
early.to_csv('../output/2018-09-12_output_table_for_maria_early_spermatocyte.csv.gz', compression='gzip')

## Maria asked for chromosomal locations

Maria responded with an email and asked if I could resend the files but with chromosomal locations.

In [11]:
# import locs and munge on
locs = pd.read_parquet('../output/paper_submission/gene_locations.parquet', columns=['chrom', 'pos'])
gonia_w_locs = gonia.join(locs).set_index(['chrom', 'pos'], append=True)
early_w_locs = early.join(locs).set_index(['chrom', 'pos'], append=True)

In [17]:
# export the data
gonia_w_locs.to_csv('../output/2018-09-13_output_table_for_maria_gonia_w_locs.csv.gz', compression='gzip')
early_w_locs.to_csv('../output/2018-09-13_output_table_for_maria_early_spermatocyte_w_locs.csv.gz', compression='gzip')